In [1]:
! pip install kaggle

In [2]:
! cd sample_data;mkdir kaggle

In [3]:
! ls -lrt


total 8
drwx------ 5 root root 4096 Jan 26 07:06 drive
drwxr-xr-x 1 root root 4096 Jan 26 07:07 sample_data


In [7]:
! chmod 600 /content/sample_data/kaggle/kaggle.json

In [5]:
mkdir /root/.kaggle

In [8]:
cp /content/sample_data/kaggle/kaggle.json /root/.kaggle/

In [9]:
! kaggle competitions download g-research-crypto-forecasting

  0% 0.00/59.0 [00:00<?, ?B/s]
100% 59.0/59.0 [00:00<00:00, 48.0kB/s]
  0% 0.00/660k [00:00<?, ?B/s]
100% 660k/660k [00:00<00:00, 43.2MB/s]
  0% 0.00/444 [00:00<?, ?B/s]
100% 444/444 [00:00<00:00, 417kB/s]
 99% 1.01G/1.02G [00:37<00:00, 30.1MB/s]
100% 1.02G/1.02G [00:38<00:00, 28.8MB/s]
  0% 0.00/406 [00:00<?, ?B/s]
100% 406/406 [00:00<00:00, 419kB/s]
 88% 90.0M/103M [00:00<00:00, 95.5MB/s]
100% 103M/103M [00:00<00:00, 118MB/s]  
  0% 0.00/5.78k [00:00<?, ?B/s]
100% 5.78k/5.78k [00:00<00:00, 5.04MB/s]


In [10]:
! unzip /content/train.csv.zip

Archive:  /content/train.csv.zip
  inflating: train.csv               


In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
import pprint
RANDOM_STATE = 42

In [12]:
crypto_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/example_test.csv')
if True:
    crypto_df_train = crypto_df[crypto_df['timestamp'] < test_df['timestamp'].min()]




In [13]:
crypto_df_train['datetime'] = pd.to_datetime(crypto_df_train['timestamp'], unit='s')
crypto_df_train = crypto_df_train.set_index('datetime')
crypto_df_train = crypto_df_train[(crypto_df_train.index.year == 2021) & (crypto_df_train.index.month > 5)]
crypto_df_train = crypto_df_train.set_index('timestamp')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
del crypto_df

In [15]:
crypto_df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 241875 entries, 1622505660 to 1623542340
Data columns (total 9 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Asset_ID  241875 non-null  int64  
 1   Count     241875 non-null  float64
 2   Open      241875 non-null  float64
 3   High      241875 non-null  float64
 4   Low       241875 non-null  float64
 5   Close     241875 non-null  float64
 6   Volume    241875 non-null  float64
 7   VWAP      241875 non-null  float64
 8   Target    241826 non-null  float64
dtypes: float64(8), int64(1)
memory usage: 18.5 MB


In [17]:
Test.index.min()

1623541500

In [18]:
Test = crypto_df_train.iloc[-200:]
Train = crypto_df_train[crypto_df_train.index < Test.index.min()]

In [19]:
# Two new features from the competition tutorial
def upper_shadow(df):
    return df['High'] - np.maximum(df['Close'], df['Open'])

def lower_shadow(df):
    return np.minimum(df['Close'], df['Open']) - df['Low']

# A utility function to build features from the original df
# It works for rows to, so we can reutilize it.
def get_features(df):
    df_feat = df[['Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP']].copy()
    df_feat['Upper_Shadow'] = upper_shadow(df_feat)
    df_feat['Lower_Shadow'] = lower_shadow(df_feat)
    return df_feat

def get_Xy_and_model_for_asset(df_train, asset_id):
    df = df_train[df_train["Asset_ID"] == asset_id]
    
    # TODO: Try different features here!
    df_proc = get_features(df)
    df_proc['y'] = df['Target']
    df_proc = df_proc.dropna(how="any")
    
    X = df_proc.drop("y", axis=1)
    y = df_proc["y"]
    
    # TODO: Try different models here!
    model = LGBMRegressor(n_estimators=100)
    model.fit(X, y)
    return X, y, model

In [20]:
x_train = get_features(Train)
y_train = Train['Target']

x_test = get_features(Test)
y_test = Test['Target']

In [ ]:


#Set the minimum error arbitrarily large
min = 99999999999999999999999 
count = 0 #Used for keeping track of the iteration number
#How many runs to perform using randomly selected hyperparameters
iterations = 50
for i in range(iterations):
    print('iteration number', count)
    count += 1 #increment count
    try:
        d_train = lgb.Dataset(x_train, label=y_train) #Load in data
        params = {} #initialize parameters
        params['learning_rate'] = np.random.uniform(0, 1)
        params['boosting_type'] = np.random.choice(['gbdt', 'dart', 'goss'])
        params['objective'] = 'regression'
        params['metric'] = 'mae'
        params['sub_feature'] = np.random.uniform(0, 1)
        params['num_leaves'] = np.random.randint(20, 300)
        params['min_data'] = np.random.randint(10, 100)
        params['max_depth'] = np.random.randint(5, 200)
        iterations = np.random.randint(10, 10000)
        print(params, iterations)
        #Train using selected parameters
        clf = lgb.train(params, d_train, iterations)
        y_pred=clf.predict(x_test) #Create predictions on test set
        mae=mean_absolute_error(y_pred,y_test)
        print('MAE:', mae)
        if mae < min:
            min = mae
            pp = params 
    except: #in case something goes wrong
        print('failed with')
        print(params)
print("*" * 50)
print('Minimum is: ', min)
print('Used params', pp)

iteration number 0
{'learning_rate': 0.43507432627962683, 'boosting_type': 'goss', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.09700303751587491, 'num_leaves': 255, 'min_data': 41, 'max_depth': 22} 8774
MAE: 0.002582376490095529
iteration number 1
{'learning_rate': 0.04945819653484207, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.12567008013847558, 'num_leaves': 160, 'min_data': 93, 'max_depth': 136} 8067
MAE: 0.002443787387517331
iteration number 2
{'learning_rate': 0.1947362760311243, 'boosting_type': 'goss', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.4757010781030019, 'num_leaves': 131, 'min_data': 41, 'max_depth': 30} 6138
MAE: 0.0025510543405400277
iteration number 3
{'learning_rate': 0.9704882161836457, 'boosting_type': 'goss', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.03633898368930211, 'num_leaves': 142, 'min_data': 97, 'max_depth': 129} 5523
MAE: 0.043506643560891775
iteration number